In [1]:
#!git clone https://github.com/kimdonggyu2008/tacotron_tts_practice
%cd tacotron_tts_practice
!git submodule init; git submodule update

[Errno 2] No such file or directory: 'tacotron_tts_practice'
/kaggle/working
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [2]:
import pandas as pd


metadata = pd.read_csv("../../input/johnoliver/metadata.csv")
metadata = metadata[metadata['include']]

total = len(metadata)
split = 0.9
train = int(total * split)

train_metadata = metadata[:train]
val_metadata = metadata[train:]

with open('/kaggle/working/tacotron_tts_practice/filelists/audio_text_train_filelist.txt', 'w') as f:
    for _, (index, _, _, _, text, _) in train_metadata.iterrows():
        filepath = '../../input/johnoliver/wav/%05d.wav' % index
        f.write(filepath + '|' + text + '\n')

with open('/kaggle/working/tacotron_tts_practice/filelists/audio_text_test_filelist.txt', 'w') as f:
    for _, (index, _, _, _, text, _) in val_metadata.iterrows():
        filepath = '../../input/johnoliver/wav/%05d.wav' % index
        f.write(filepath + '|' + text + '\n')

FileNotFoundError: [Errno 2] No such file or directory: '../../input/johnoliver/metadata.csv'

In [ ]:
df = pd.read_csv("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt", sep="|", names=["path", "text"])

In [ ]:
with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt", "r") as f:
    lines = f.readlines()

with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt", "w") as f:
    for line in lines:
        line = line.replace('DUMMY', '/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs')
        f.write(line)

In [ ]:
with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_val_filelist.txt", "r") as f:
    lines = f.readlines()
with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_val_filelist.txt", "w") as f:
    for line in lines:
        line = line.replace('DUMMY', '/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs')
        f.write(line)

In [ ]:
with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_test_filelist.txt", "r") as f:
    lines = f.readlines()
with open("/kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_test_filelist.txt", "w") as f:
    for line in lines:
        line = line.replace('DUMMY', '/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs')
        f.write(line)

In [ ]:
!mv /kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_train_filelist.txt /kaggle/working/tacotron_tts_practice/filelists/audio_text_train_filelist.txt

In [ ]:
!mv /kaggle/working/tacotron_tts_practice/filelists/ljs_audio_text_test_filelist.txt /kaggle/working/tacotron_tts_practice/filelists/audio_text_test_filelist.txt

In [ ]:
!ls tacotron_tts_practice/filelists

In [ ]:
!pip install tensorflow==2.17
!pip install unidecode
!pip install inflect

In [ ]:
!python --version

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA

In [ ]:
!apt-get update --fix-missing
!apt-get install sox -y
!soxi /kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs/LJ001-0001.wav

In [ ]:
#!python train.py --output_directory=outdir --log_directory=logdir -c tacotron2_statedict.pt --warm_start --n_gpus=2 --rank=0 --group_name='group_name' --hparams sampling_rate=22050:iters_per_checkpoint=100

In [ ]:
!python -m pip install spacy

In [ ]:
!pip install inflect==5.0.1

In [ ]:
from multiprocessing import Pool
import matplotlib.pylab as plt
%matplotlib inline
from tqdm.notebook import tqdm
import IPython.display as ipd
from time import time, sleep
import scipy.io.wavfile
import numpy as np
import matplotlib
import torch
import sys
sys.path.append('waveglow/')

In [ ]:
%cp /kaggle/working/WaveGlow/denoiser.py /opt/conda/lib/python3.10/site-packages/denoiser/


In [ ]:
from audio_processing import griffin_lim
from layers import TacotronSTFT, STFT
from hparams import create_hparams
from text import text_to_sequence

In [ ]:
from denoiser import Denoiser
from train import *
from model import *
import tensorflow as tf

In [ ]:
from denoiser import Denoiser
from train import load_model
from model import Tacotron2

In [ ]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower', interpolation='none')

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050

In [ ]:
# checkpoint = int(input("Enter steps at latest checkpoint: "))
checkpoint_path = f"outdir/checkpoint_1000"
# checkpoint_path = "/kaggle/working/tacotron2/tacotron2_statedict.pt"
model =load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

In [ ]:
!gdown https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF

In [ ]:
waveglow_path = 'waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [ ]:
def synthesize(word, n=5, cleaners=['english_cleaners'], sigma=0.666, strength=0.01):
    sequence = np.array(text_to_sequence(word, cleaners))[None, :]
    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
    with torch.no_grad():
        mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    audio = waveglow.infer(mel_outputs_postnet, sigma=sigma)
    audio_denoised = denoiser(audio, strength=strength)[:, 0]
    audio = audio[0].data.cpu().numpy().tolist()
    mel_outputs = mel_outputs.float().data.cpu().numpy()[0]
    mel_outputs_postnet = mel_outputs_postnet.float().data.cpu().numpy()[0]
    alignments = alignments.float().data.cpu().numpy()[0]
    return audio, mel_outputs, mel_outputs_postnet, alignments

In [ ]:
tests = [
         "Scientists at the CERN laboratory say they have discovered a new particle."]